##### Copyright 2024 Google LLC.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Search grouding with Gemini 1.5

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/search_grounding_for_research_report.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

In this tutorial you are going to leverage the grounding with Google Search capability of the Gemini 1.5 model to write a company report.

You may be asking, why does one need to use the search tool for this purpose? Well, as you may be aware, today's business world evolves very fast and LLMs generally are not trained frequently enough to capture the latest updates. Luckily Google search comes to the rescue. Google search is built to provide accurate and nearly realtime information and can help us fulfill this task nicely.

Note that the latest [search tool](https://github.com/google-gemini/cookbook/blob/main/gemini-2/search_tool.ipynb) that comes with Gemini 2.0 is much easier to use and should be prioritized over search grounding in Gemini 1.5 models.

## Setup

### Install SDK

The new **[Google Gen AI SDK](https://ai.google.dev/gemini-api/docs/sdks)** provides programmatic access to Gemini 2.0 (and previous models) using both the [Google AI for Developers](https://ai.google.dev/gemini-api/docs) and [Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/overview) APIs. With a few exceptions, code that runs on one platform will run on both. This means that you can prototype an application using the Developer API and then migrate the application to Vertex AI without rewriting your code.

More details about this new SDK on the [documentation](https://ai.google.dev/gemini-api/docs/sdks) or in the [Getting started](../gemini-2/get_started.ipynb) notebook.

In [2]:
!pip install -U -q google-genai

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [3]:
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

### Import the libraries

In [4]:
from google import genai
import json
from IPython.display import display, HTML, Markdown

### Initialize SDK client

With the new SDK you now only need to initialize a client with you API key (or OAuth if using [Vertex AI](https://link_to_vertex_AI)). The model is now set in each call.

In [5]:
client = genai.Client(api_key=GOOGLE_API_KEY)

### Select a model

Search tool is a new feature in the Gemini 2.0 model that automatically retrieves accurate and grounded artifacts from the web for developers to further process. Unlike the search grounding in the Gemini 1.5 models, you do not need to set the dynamic retrieval threshold.

For more information about all Gemini models, check the [documentation](https://ai.google.dev/gemini-api/docs/models/gemini) for extended information on each of them.


In [6]:
MODEL = "gemini-1.5-flash"

## Write the report with Gemini 1.5 Flash

### Select a target company to research

Next you will use Alphabet as an example research target.

In [7]:
COMPANY = 'Alphabet'

### Plan for searches
You use the Gemini model to plan for 5 key queries to send to Google search to get the latest updates on the company. You can of course use as many queries as you want.

In [8]:
from google import genai
from google.genai.types import GenerateContentConfig

result = client.models.generate_content(
    model=MODEL,
    contents=[f"""You are an analyst that conducts comprehensive company research.
    You rely heavily on Google search for the latest company news, updates and metrics to write research reports.
    For {COMPANY}, please generate a list of 5 key queries you'd like to send to Google to get accurate answers before you write the report.
    Return queries only; no other information needed."""],
    config=GenerateContentConfig(
        response_mime_type='application/json',
    )
)

Here are the 5 queries to be used.

In [9]:
search_queries = json.loads(result.candidates[0].content.parts[0].text)['queries']
print(search_queries)

['Alphabet Inc. latest quarterly earnings report', 'Alphabet Inc. stock price and performance analysis', "Recent news and updates on Alphabet Inc.'s AI initiatives", 'Competitive landscape analysis for Alphabet Inc. in the search engine market', "Alphabet Inc.'s recent investments and acquisitions"]


### Send queries to Google search and save the results

In [10]:
chat = client.chats.create(model=MODEL, config={'tools': [{'google_search_retrieval': {}}]})

query_answer = {}
search_suggestions = []
for query in search_queries:
  print(f'Searhing for query: {query}')
  r = chat.send_message(query)
  all_parts = []
  for part in r.candidates[0].content.parts:
    all_parts.extend([v for k, v in part.model_dump().items() if v is not None])
  query_answer[query] = all_parts
  grounding_metadata = r.candidates[0].grounding_metadata
  if grounding_metadata and grounding_metadata.search_entry_point:
    search_suggestions.append(grounding_metadata.search_entry_point.rendered_content)

Searhing for query: Alphabet Inc. latest quarterly earnings report
Searhing for query: Alphabet Inc. stock price and performance analysis
Searhing for query: Recent news and updates on Alphabet Inc.'s AI initiatives
Searhing for query: Competitive landscape analysis for Alphabet Inc. in the search engine market
Searhing for query: Alphabet Inc.'s recent investments and acquisitions


### Generate the final report

Next you define a couple of helper functions to better display the final report.

In [11]:
def show_json(obj):
  print(json.dumps(obj.model_dump(exclude_none=True), indent=2))

def show_report(r, search_suggestions = []):
  parts = r.candidates[0].content.parts
  if parts is None:
    finish_reason = r.candidates[0].finish_reason
    print(f'{finish_reason=}')
    return
  for part in r.candidates[0].content.parts:
    if part.text:
      display(Markdown(part.text.replace('$', r'\$')))  # Escape $ signs for better MD rendering
    elif part.executable_code:
      display(Markdown(f'```python\n{part.executable_code.code}\n```'))
    else:
      show_json(part)

  if len(search_suggestions) > 0:
    for s in search_suggestions:
      display(HTML(s))

Now you can ask the Gemini model to put together the final research report based on all the previous search results.

In [12]:
final_report = client.models.generate_content(
    model=MODEL,
    contents=[f"""You are an analyst that conducts company research.
    You are working on a company report on {COMPANY} and have gathered the following comprehensive information based on various Google searches.

    {json.dumps(query_answer)}

    Based on all these information, please write a concise company report."""],
)

Display the final report.

In [13]:
show_report(final_report, search_suggestions)

**Alphabet Inc. Company Report - December 17, 2024**

**Executive Summary:** Alphabet Inc. (GOOGL) delivered record Q3 2024 earnings, exceeding analyst expectations across revenue (\$88.3B) and profit (\$26.3B), driven by strong performance in Search, Cloud, and YouTube.  The launch of Gemini 2.0 and its integration across products significantly contributed to growth.  While the stock price surged post-earnings,  it remains below its 52-week high, reflecting previous volatility and concerns around antitrust investigations and substantial AI investments. Analyst sentiment is positive, with a consensus "Strong Buy" rating and a 12-month price target of \$206.95.  However,  longer-term performance shows a slight decline, highlighting the dynamic competitive landscape.


**Financial Performance:**

* Q3 2024: Record revenue and profit exceeding analyst projections.  Strong advertising revenue growth.
* Stock Price:  Recent gains following strong Q3 results, but mixed performance over the past year; currently trading around \$195-\$198, with variations across sources.

**Key Strategic Initiatives:**

* **AI Leadership:** Aggressive expansion of AI capabilities with Gemini 2.0, impacting various Google products and attracting partnerships (e.g., Malaysian government).  AI is used internally for approximately 25% of new software development.
* **Acquisitions and Investments:**  A mixed approach, involving both significant (Mandiant, potential Wiz acquisition) and smaller strategic acquisitions across cybersecurity, AI, and other sectors.  Focus on strengthening existing businesses and expanding into new markets. The failed HubSpot acquisition highlights the risks associated with large-scale deals.


**Competitive Landscape:**

* **Dominant Market Share:** Google Search maintains over 90% market share.
* **Intense Competition:**  Faces competition from Microsoft (Bing), smaller search engines, vertical search engines, and emerging AI companies.
* **Competitive Pressures:**  Market share battles, technological innovation (especially AI), regulatory scrutiny, and advertising revenue dependence are key challenges.


**Risks and Uncertainties:**

* **Antitrust concerns and regulatory pressures:** Potential impact on market dominance and business practices.
* **Heavy AI investment:**  While promising, the substantial investment in AI carries inherent risks and uncertainties regarding future profitability.
* **Competition:** Maintaining market dominance in a rapidly evolving technological landscape.


**Outlook:**

Alphabet's strong Q3 performance and significant AI investments position it for continued growth.  However, navigating the competitive landscape and addressing regulatory concerns will be crucial.  The potential acquisition of Wiz could be a significant milestone.  Analyst sentiment remains positive, predicting further stock price appreciation, although the recent past suggests some volatility is to be expected.  Further monitoring of market conditions and the evolution of the AI landscape is essential.


As you can see, the Gemini 1.5 model is able to write an accurate and well-structured research report for us. All the information in the report is factual and up-to-date.

Note that although Gemini 1.5 Flash is able to finish this task, it still requires 3 manual steps. With Gemini 2.0, you can accomplish the same with one API call. Please check out our search tool for research tool tutorial for details.

## Next Steps
### Useful API references:

* [Grounding with Google Search](https://ai.google.dev/gemini-api/docs/grounding?lang=python)